In [1]:
import sqlite3
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas_profiling import ProfileReport
import sklearn as sk
import dtale

In [2]:
# df = pd.read_csv("donors.csv", low_memory=False).drop(columns="Unnamed: 0")
# df.to_pickle("donors.pkl")
# sample = df.iloc[:1000, :].copy()
# sample.to_pickle("sample.pkl")

In [23]:
df = pd.read_pickle("donors.pkl")
sample = pd.read_pickle("sample.pkl")

# EDA DONE

In [24]:
# The following variables indicate the number of
# known times the donor has responded to other
# types of mail order offers.

groupMailOffers = sample.loc[:, "MBCRAFT":"PUBOPP"].columns.to_list()

# dtale.show(sample[mailOrderOffers])

for col in groupMailOffers:
    sample[col] = sample[col].fillna(0).astype(np.int16)

In [25]:
searchName = "MDMAUD"
namesToGroup = [col for col in sample.columns.to_list() if searchName in col]
print(namesToGroup)

['MDMAUD', 'MDMAUD_R', 'MDMAUD_F', 'MDMAUD_A']


In [26]:
for name in namesToGroup:
    setattr(sample, name, sample[name].str.replace("X", "").replace("", np.nan))

In [27]:
# MAJOR
namesToGroup = ["MAJOR"] + namesToGroup
sample.MAJOR.replace({"X": True, " ": False}, inplace=True)

In [28]:
# RFA_2
groupRFA_2 = ["RFA_2", "RFA_2R", "RFA_2F", "RFA_2A"]
namesToGroup += groupRFA_2
sample[namesToGroup]

,MAJOR,MDMAUD,MDMAUD_R,MDMAUD_F,MDMAUD_A,RFA_2,RFA_2R,RFA_2F,RFA_2A
0,False,NaN,NaN,NaN,NaN,L4E,L,4,E
1,False,NaN,NaN,NaN,NaN,L2G,L,2,G
2,False,NaN,NaN,NaN,NaN,L4E,L,4,E
3,False,NaN,NaN,NaN,NaN,L4E,L,4,E
4,False,NaN,NaN,NaN,NaN,L2F,L,2,F
...,...,...,...,...,...,...,...,...,...
995,False,NaN,NaN,NaN,NaN,L2F,L,2,F
996,False,NaN,NaN,NaN,NaN,L1G,L,1,G
997,False,NaN,NaN,NaN,NaN,L1F,L,1,F
998,False,NaN,NaN,NaN,NaN,L4D,L,4,D


In [29]:
df.RFA_2R.describe()

count     95412
unique        1
top           L
freq      95412
Name: RFA_2R, dtype: object

In [30]:
pd.api.types.CategoricalDtype()

CategoricalDtype(categories=None, ordered=False)

In [31]:
for c in namesToGroup:
    sample[c] = pd.Categorical(sample[c])

In [32]:
sample[namesToGroup].dtypes

MAJOR       category
MDMAUD      category
MDMAUD_R    category
MDMAUD_F    category
MDMAUD_A    category
RFA_2       category
RFA_2R      category
RFA_2F      category
RFA_2A      category
dtype: object

In [33]:
toReorder = ["ADATE", "RFA", "RDATE", "RAMNT"]
reorderedCols = []
for i in range(3, 25):
    reorderedCols += [str(col)+"_"+str(i) for col in toReorder]
reorderedCols = ["ADATE_2"] + groupRFA_2 + reorderedCols

dictIdx = {col: sample.columns.get_loc(col) for col in reorderedCols}

temp = sample[dictIdx.keys()].copy()
sample.drop(columns=dictIdx.keys(), inplace=True)

for key in dictIdx.keys():
    sample[key] = temp[key]

In [34]:
# Converting date columns into datetimes
for c in (dateNames := [c for c in sample.columns.values if ("DATE" in c)]):
    sample[c] = pd.to_datetime(sample[c], infer_datetime_format=True)
    
sample.DOB = pd.to_datetime(sample.DOB)

In [35]:
# Always keep this cell last before EDA WIP
sample.to_pickle("sample_eda_done.pkl")

# EDA WIP

In [42]:
# restart work from here
sample = pd.read_pickle("sample_eda_done.pkl")

In [44]:
# Drop entries where
# df[df.ODATEDW < df.DOB]

,ODATEDW,OSOURCE,TCODE,STATE,ZIP,MAILCODE,PVASTATE,DOB,NOEXCH,RECINHSE,...,AVGGIFT,CONTROLN,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,GEOCODE2
919,2006-01-01,MBC,2,MO,63366,,,2010-09-01,0,,...,3.400000,46266,1,L,4,D,X,X,X,A
924,2011-01-01,L01,2,MO,65801,,,2011-02-01,0,,...,3.166667,104202,1,L,4,D,X,X,X,B
1350,2006-01-01,DRK,1,FL,32746,,,2010-01-01,0,,...,7.736842,34931,1,L,3,E,X,X,X,B
1426,2006-01-01,MBC,2,WI,53006,,,2010-07-01,0,,...,7.181818,77118,0,L,2,E,X,X,X,C
1526,2006-01-01,AMB,0,NC,27889,,,2010-01-01,0,X,...,5.258065,7188,0,L,2,E,X,X,X,D


In [52]:
df["AGE_2020"] = np.int16(2020 - pd.to_datetime(df.DOB).dt.year)

In [53]:
df["AGE_2020"].describe()

count    95412.000000
mean        46.259632
std         30.426620
min          0.000000
25%          0.000000
50%         53.000000
75%         72.000000
max         90.000000
Name: AGE_2020, dtype: float64

In [68]:
df[(df.AGE_2020  == 3)]

,ODATEDW,OSOURCE,TCODE,STATE,ZIP,MAILCODE,PVASTATE,DOB,NOEXCH,RECINHSE,...,CONTROLN,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,GEOCODE2,AGE_2020
22984,2010-01-01,,2,KY,40476,,,2017-06-01,0,,...,205,0,L,1,F,X,X,X,C,3
31300,2006-01-01,EPL,2,AZ,85351,,,2017-04-01,0,,...,137263,1,L,3,G,X,X,X,A,3
40565,2011-01-01,SYN,0,IL,60131,,,2017-08-01,0,,...,89765,1,L,3,F,X,X,X,A,3
52252,2006-01-01,DNA,2,IL,60201,,,2017-10-01,0,X,...,8080,1,L,2,G,X,X,X,A,3
60753,2006-01-01,ENQ,0,FL,33702,,,2017-10-01,0,,...,37552,0,L,4,D,X,X,X,A,3
71837,2006-01-01,EPL,2,MS,38732,,,2017-01-01,0,X,...,188455,1,L,2,F,X,X,X,D,3
88362,2007-01-01,FCR,2,CA,91020,,,2017-01-01,0,X,...,188328,0,L,1,F,X,X,X,A,3
94452,2006-01-01,HOL,2,CO,80911,,,2017-08-01,0,,...,131660,0,L,3,E,X,X,X,B,3


In [75]:
df[(df.TCODE > 120) & (df.TCODE < 220)].RAMNTALL

7390     126.0
7546      25.0
11400     87.0
25979     75.0
26620     46.0
42487    347.0
45811    170.0
57962    187.0
62336     57.0
91047    120.0
95105    228.0
Name: RAMNTALL, dtype: float64